In [1]:
import sklearn.datasets as datasets
import torch
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
df = datasets.fetch_california_housing()
X = df.data
y = df.target

X, y

(array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
           37.88      , -122.23      ],
        [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
           37.86      , -122.22      ],
        [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
           37.85      , -122.24      ],
        ...,
        [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
           39.43      , -121.22      ],
        [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
           39.43      , -121.32      ],
        [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
           39.37      , -121.24      ]]),
 array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894]))

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=13)

#### Создать Dataset для загрузки данных

In [41]:
class HousingDataset(torch.utils.data.Dataset):
   
    def __init__(self, data, target):
        self._data = data
        self._target = target

    def __len__(self):
        return len(self._data)

    def __getitem__(self, idx):
        return self._data[idx], self._target[idx]

In [65]:
train_ds = HousingDataset(X_train, y_train)

#### Обернуть его в Dataloader

In [71]:
batch_size = 10

In [133]:
train_loader = torch.utils.data.DataLoader(train_ds,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)
# при num_workers > 0 выдаёт ошибку

In [134]:
#for i, batch in enumerate(train_loader):
#    print(i, batch)

#### Написать архитектуру сети, которая предсказывает стоимость недвижимости. Сеть должна включать BatchNorm слои и Dropout (или НЕ включать, но нужно обосновать)

In [163]:
features_len = len(df['feature_names'])

In [162]:
class MLP(nn.Module):
  '''
    Multilayer Perceptron for regression.
  '''
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Linear(features_len, 64),
      nn.ReLU(),
      nn.Linear(64, 32),
      nn.ReLU(),
      nn.Linear(32, 1)
    )


  def forward(self, x):
    '''
      Forward pass
    '''
    return self.layers(x)

In [172]:
net = MLP()

loss_function = nn.L1Loss()
optimizer = torch.optim.Adam(net.parameters(), lr=1e-4)

In [175]:
from tqdm import tqdm

for epoch in tqdm(range(10)):  
    running_loss = 0.0
    for i, data in enumerate(train_loader, 1):
        inputs, targets = data[0].float(), data[1].float()
        targets = targets.reshape((targets.shape[0], 1))

        # обнуляем градиент
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = loss_function(outputs, targets)
        loss.backward()
        optimizer.step()

        # выводим статистику о процессе обучения
        running_loss += loss.item()
        if i % 300 == 0:    # печатаем каждые 300 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 300))
            running_loss = 0.0

print('Training is finished!')

  0%|          | 0/10 [00:00<?, ?it/s]

[1,   301] loss: 0.961
[1,   601] loss: 0.984
[1,   901] loss: 0.930
[1,  1201] loss: 0.898


 10%|█         | 1/10 [00:00<00:08,  1.02it/s]

[1,  1501] loss: 0.921
[2,   301] loss: 0.884
[2,   601] loss: 0.851
[2,   901] loss: 0.838
[2,  1201] loss: 0.852


 20%|██        | 2/10 [00:01<00:07,  1.01it/s]

...

[8,  1501] loss: 0.660
[9,   301] loss: 0.655
[9,   601] loss: 0.653
[9,   901] loss: 0.642
[9,  1201] loss: 0.682
[9,  1501] loss: 0.634


 90%|█████████ | 9/10 [00:09<00:01,  1.03s/it]

[10,   301] loss: 0.665
[10,   601] loss: 0.681
[10,   901] loss: 0.653
[10,  1201] loss: 0.697


100%|██████████| 10/10 [00:10<00:00,  1.02s/it]

[10,  1501] loss: 0.634
Training is finished!


#### Вариант 2

In [182]:
import torch.nn.functional as F
import torch.nn as nn

class Perceptron(nn.Module):
    def __init__(self, input_dim, output_dim, activation="relu"):
        super(Perceptron, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)
        self.activation = activation
        
    def forward(self, x):
        x = self.fc(x)
        if self.activation=="relu":
            return F.relu(x)
        if self.activation=="sigmoid":
            return F.sigmoid(x)
        raise RuntimeError
        

class FeedForward(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(FeedForward, self).__init__()
        #self.bn1 = nn.BatchNorm1d(input_dim)
        self.fc1 = Perceptron(input_dim, hidden_dim)
        #self.bn2 = nn.BatchNorm1d(hidden_dim)
        #self.dp = nn.Dropout(0.25)
        self.fc2 = Perceptron(hidden_dim, 1, "relu")
        
    def forward(self, x):
        #x = self.bn1(x)
        x = self.fc1(x)
        #x = self.bn2(x)
        #x = self.dp(x)
        x = self.fc2(x)
        return x

In [183]:
net = FeedForward(features_len, 1)

optimizer = torch.optim.SGD(net.parameters(), lr=0.01)
# criterion = nn.CrossEntropyLoss() # для классификации
criterion = nn.L1Loss() # для регрессии

In [186]:
from tqdm import tqdm

for epoch in tqdm(range(10)):  
    running_loss = 0.0
    for i, data in enumerate(train_loader, 1):
        inputs, targets = data[0].float(), data[1].float()
        targets = targets.reshape((targets.shape[0], 1))

        # обнуляем градиент
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = loss_function(outputs, targets)
        loss.backward()
        optimizer.step()

        # выводим статистику о процессе обучения
        running_loss += loss.item()
        if i % 300 == 0:    # печатаем каждые 300 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 300))
            running_loss = 0.0

print('Training is finished!')

  0%|          | 0/10 [00:00<?, ?it/s]

[1,   301] loss: 2.043
[1,   601] loss: 2.098
[1,   901] loss: 2.053
[1,  1201] loss: 2.056


 10%|█         | 1/10 [00:00<00:06,  1.46it/s]

[1,  1501] loss: 2.074
[2,   301] loss: 2.076
[2,   601] loss: 2.055
[2,   901] loss: 2.043


 20%|██        | 2/10 [00:01<00:05,  1.47it/s]

...

[8,  1201] loss: 2.094
[8,  1501] loss: 2.067
[9,   301] loss: 2.054
[9,   601] loss: 2.067
[9,   901] loss: 2.062
[9,  1201] loss: 2.085


 90%|█████████ | 9/10 [00:05<00:00,  1.64it/s]

[9,  1501] loss: 2.064
[10,   301] loss: 2.056
[10,   601] loss: 2.048
[10,   901] loss: 2.071


100%|██████████| 10/10 [00:06<00:00,  1.58it/s]

[10,  1201] loss: 2.085
[10,  1501] loss: 2.076
Training is finished!


#### Вариант 3

In [188]:
import torch.nn.functional as F
import torch.nn as nn

class Perceptron(nn.Module):
    def __init__(self, input_dim, output_dim, activation="relu"):
        super(Perceptron, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)
        self.activation = activation
        
    def forward(self, x):
        x = self.fc(x)
        if self.activation=="relu":
            return F.relu(x)
        if self.activation=="sigmoid":
            return F.sigmoid(x)
        raise RuntimeError
        

class FeedForward(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(FeedForward, self).__init__()
        self.bn1 = nn.BatchNorm1d(input_dim)
        self.fc1 = Perceptron(input_dim, hidden_dim)
        self.bn2 = nn.BatchNorm1d(hidden_dim)
        self.dp = nn.Dropout(0.25)
        self.fc2 = Perceptron(hidden_dim, 1, "relu")
        
    def forward(self, x):
        x = self.bn1(x)
        x = self.fc1(x)
        x = self.bn2(x)
        x = self.dp(x)
        x = self.fc2(x)
        return x

In [189]:
net = FeedForward(features_len, 1)

optimizer = torch.optim.SGD(net.parameters(), lr=0.01)
# criterion = nn.CrossEntropyLoss() # для классификации
criterion = nn.L1Loss() # для регрессии

In [190]:
from tqdm import tqdm

for epoch in tqdm(range(10)):  
    running_loss = 0.0
    for i, data in enumerate(train_loader, 1):
        inputs, targets = data[0].float(), data[1].float()
        targets = targets.reshape((targets.shape[0], 1))

        # обнуляем градиент
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = loss_function(outputs, targets)
        loss.backward()
        optimizer.step()

        # выводим статистику о процессе обучения
        running_loss += loss.item()
        if i % 300 == 0:    # печатаем каждые 300 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 300))
            running_loss = 0.0

print('Training is finished!')

  0%|          | 0/10 [00:00<?, ?it/s]

[1,   301] loss: 1.129
[1,   601] loss: 0.883
[1,   901] loss: 0.836
[1,  1201] loss: 0.720


 10%|█         | 1/10 [00:01<00:14,  1.58s/it]

[1,  1501] loss: 0.676
[2,   301] loss: 0.644
[2,   601] loss: 0.671
[2,   901] loss: 0.659
[2,  1201] loss: 0.653


 20%|██        | 2/10 [00:03<00:12,  1.59s/it]

...

[8,  1501] loss: 0.661
[9,   301] loss: 0.674
[9,   601] loss: 0.656
[9,   901] loss: 0.645
[9,  1201] loss: 0.669


 90%|█████████ | 9/10 [00:13<00:01,  1.50s/it]

[9,  1501] loss: 0.671
[10,   301] loss: 0.648
[10,   601] loss: 0.684
[10,   901] loss: 0.637
[10,  1201] loss: 0.659


100%|██████████| 10/10 [00:15<00:00,  1.52s/it]

[10,  1501] loss: 0.660
Training is finished!
